### Infosys 722 - Iteration 4 BDAS (Steps 1-8)
### Submitted by: Sym Concepcion
### UPI: scon497, studentID: 829972794

In [1]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')

In [2]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('722Basics').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/08 03:06:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Let's read in the data. There are 2 sources so we can integrate later. 
# Note that it's in the format of csv
df1 = spark.read.option("header", True).csv('dataset/data_ncr.csv')
df2 = spark.read.option("header", True).csv('dataset/data_therest.csv')

In [4]:
# df.show()
print("Files read into structure of",type(df1))
print("First dataset (NCR):", df1.count(), "x", len(df1.columns))
print("Second dataset (Rest of the regions):", df2.count(), "x", len(df2.columns))

Files read into structure of <class 'pyspark.sql.dataframe.DataFrame'>
First dataset (NCR): 4130 x 60
Second dataset (Rest of the regions): 37414 x 60


In [5]:
# integrate the 2 sources
# code adapted from https://www.geeksforgeeks.org/merge-two-dataframes-in-pyspark/
import functools 
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

statsdata = unionAll([df1, df2])

In [7]:
print("Combined dataset:", statsdata.count(), "x", len(statsdata.columns))
statsdata.printSchema()

Combined dataset: 41544 x 60
root
 |-- Total Household Income: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Total Food Expenditure: string (nullable = true)
 |-- Main Source of Income: string (nullable = true)
 |-- Agricultural Household indicator: string (nullable = true)
 |-- Bread and Cereals Expenditure: string (nullable = true)
 |-- Total Rice Expenditure: string (nullable = true)
 |-- Meat Expenditure: string (nullable = true)
 |-- Total Fish and  marine products Expenditure: string (nullable = true)
 |-- Fruit Expenditure: string (nullable = true)
 |-- Vegetables Expenditure: string (nullable = true)
 |-- Restaurant and hotels Expenditure: string (nullable = true)
 |-- Alcoholic Beverages Expenditure: string (nullable = true)
 |-- Tobacco Expenditure: string (nullable = true)
 |-- Clothing, Footwear and Other Wear Expenditure: string (nullable = true)
 |-- Housing and water Expenditure: string (nullable = true)
 |-- Imputed House Rental Value: string (null

In [9]:
statsdata.head()

Row(Total Household Income='623738', Region='NCR', Total Food Expenditure='133762', Main Source of Income='Wage/Salaries', Agricultural Household indicator='0', Bread and Cereals Expenditure='21969', Total Rice Expenditure='13000', Meat Expenditure='15192', Total Fish and  marine products Expenditure='10438', Fruit Expenditure='3690', Vegetables Expenditure='5208', Restaurant and hotels Expenditure='52640', Alcoholic Beverages Expenditure='0', Tobacco Expenditure='0', Clothing, Footwear and Other Wear Expenditure='13548', Housing and water Expenditure='124170', Imputed House Rental Value='84000', Medical Care Expenditure='5784', Transportation Expenditure='19440', Communication Expenditure='19188', Education Expenditure='0', Miscellaneous Goods and Services Expenditure='41436', Special Occasions Expenditure='8200', Crop Farming and Gardening expenses='0', Total Income from Entrepreneurial Acitivites='0', Household Head Sex='Male', Household Head Age='62', Household Head Marital Status=

In [10]:
statsdata.show(10)

+----------------------+------+----------------------+---------------------+--------------------------------+-----------------------------+----------------------+----------------+-------------------------------------------+-----------------+----------------------+---------------------------------+-------------------------------+-------------------+---------------------------------------------+-----------------------------+--------------------------+------------------------+--------------------------+-------------------------+---------------------+--------------------------------------------+-----------------------------+-----------------------------------+--------------------------------------------+------------------+------------------+-----------------------------+--------------------------------------+----------------------------------------+-------------------------+------------------------------+-----------------+------------------------------+-------------------------------------

In [11]:
statsdata.describe('Total Household Income').show()

+-------+----------------------+
|summary|Total Household Income|
+-------+----------------------+
|  count|                 41544|
|   mean|    247555.58480165608|
| stddev|     286880.5226527398|
|    min|                100000|
|    max|                 99997|
+-------+----------------------+



In [12]:
statsdata.describe('Medical Care Expenditure').show()

+-------+------------------------+
|summary|Medical Care Expenditure|
+-------+------------------------+
|  count|                   41544|
|   mean|       7160.230935875216|
| stddev|      26902.002274617334|
|    min|                       0|
|    max|                    9998|
+-------+------------------------+

